<center> <img src="https://miro.medium.com/v2/resize:fit:1250/format:webp/1*QgI1t-7yJApi4vQigFgsLQ.jpeg" width=25% > </center>

<br><br>

<center> 
    <font size="6">Final Lab (Part 1): Keypoint Detection, Bag of Visual Words and Image Classification</font>
</center>
<center> 
    <font size="4">Computer Vision 1 University of Amsterdam</font> 
</center>
<center> 
    <font size="4">Due 23:59PM, 24th October, 2025 (Amsterdam time)</font> 
</center>

<br><br>

***

<br><br>

<center>

Student1 ID:  \
Student1 Name: 

Student2 ID: \
Student2 Name: 

Student3 ID: \
Student3 Name: 

( Student4 ID: \
Student4 Name: )

</center>

### **General Guidelines**

Your code must be handed in this Jupyter notebook, renamed to **StudentID1_StudentID2_StudentID3.ipynb** before the deadline by submitting it to the Canvas Final Lab (Part 1) Assignment. Please also fill out your names and IDs above.

For full credit, make sure your notebook follows these guidelines:

- Please express your thoughts **concisely**. The number of words does not necessarily correlate with how well you understand the concepts.
- Understand the problem as much as you can. When answering a question, provide evidence (qualitative and/or quantitative results, references to papers, figures, etc.) to support your arguments. Not everything might be explicitly asked for, so think about what might strengthen your arguments to make the notebook self-contained and complete.
- Tables and figures must be accompanied by a **brief** description. Add a number, a title, and, if applicable, the name and unit of variables in a table, and name and unit of axes and legends in a figure.

**Late submissions are not allowed.** Assignments submitted after the strict deadline will not be graded. In case of submission conflicts, TAs’ system clock is taken as reference. We strongly recommend submitting well in advance to avoid last-minute system failure issues.

**Environment:** Since this is a project-based assignment, you are free to use any feature descriptor and machine learning tools (e.g., K-means, SVM). You should use Python for your implementation. You are free to use any Python library for this assignment, but make sure to provide a conda environment file!

**Plagiarism Note:** Keep in mind that plagiarism (submitted materials which are not your work) is a serious offense and any misconduct will be addressed according to university regulations. This includes using generative tools such as ChatGPT.

**Ensure that you save all results/answers to the questions (even if you reuse some code).**

### **Report Preparation**

Your tasks include the following:

1. **Report Preparation:** For both parts of the final project, students are expected to prepare a report. The report should include all details on implementation approaches, analysis of results for different settings, and visualizations illustrating experiments and performance of your implementation. Grading will be based on the report, so it should be as self-contained as possible. If the report contains faulty results or ambiguities, TAs can refer to your code for clarification. Only section 10 of this notebook should **not** be included in the report.

2. **Explanation of Results:** Do not just provide numbers without explanation. Discuss different settings to show your understanding of the material and processes involved.

3. **Quantitative Evaluation:** For quantitative evaluation, you are expected to provide the results based on the mAP (mean Average Precision) metric. You should report the mAP for each experimental setup. 

4. **Qualitative Evaluation:** For qualitative evaluation, you are expected to visualize the top-5 and bottom-5 ranked test images (based on classifier confidence for the target class) per setup. Provide a figure for each experimental setup Visual elements such as charts, graphs, and plots are always useful. Keep this in mind while writing your reports.

5. **Aim:** Understand the basic Image Classification pipeline using a traditional Bag of Visual Words method.

6. **Working on Assignments:** Students should work in assigned groups for **two** weeks. Any questions can be discussed on ED.

    - **Submission:** Submit your source code and report together in a zip file (`ID1_ID2_ID3_part1.zip`). The report should be a maximum of 10 pages (single-column, including tables and figures, excluding references and appendix). Express thoughts concisely. Tables and figures must be accompanied by a description. Number them and, if applicable, name variables in tables, and label axes in figures.

7. **Hyperparameter Search:** In your experiments, remember to perform a hyperparameter search to find the optimal settings for your classifier. Clearly document the search process, the parameters you explored, and how they influenced the performance of your model.

8. **Format and Testing:** The report should be in **PDF format**, and the code in **.ipynb format**. Test that all functionality works as expected in the notebook.

### **Overview**

- [Section 1: Data Preparation (0 points)](#section-1)
- [Section 2: Keypoint Detection and Feature Extraction (3 points)](#section-2)
- [Section 3: Building the Visual Vocabulary (3 points)](#section-3)
- [Section 4: Encoding Train Image Features (3 points)](#section-4)
- [Section 5: Visualizing the Bag of Visual Words for Each Class (3 points)](#section-5)
- [Section 6: Encoding Test Image Features (0 points)](#section-6)
- [Section 7: Training the Classifiers (5 points)](#section-7)
- [Section 8: Evaluating the Classifiers (12 points)](#section-8)
- [Section 9: Hyperparameter Search (16 points)](#section-9)
- [Section 10: Using CLIP for Image Classification (5 points)](#section-10)
- [Section X: Individual Contribution Report (Mandatory)](#section-x)

### **Section 1: Data Preparation (0 points)**

The goal of this lab is to implement an image classification system that can identify objects from a given set of classes. You will perform a 5-class image classification using a bag-of-words approach ([reference](http://www.robots.ox.ac.uk/~az/icvss08_az_bow.pdf)). The classes for this task are:

1. **Frog**
2. **Automobile**
3. **Bird**
4. **Cat**
5. **Deer**

The [CIFAR-10 dataset](https://www.cs.toronto.edu/~kriz/cifar.html) will be used for this task. This dataset contains 32x32 pixel RGB images, divided into sub-directories with 5000 training images and 1000 test images for each class.

The dataset will be automatically downloaded using the code provided in this notebook. You will need to perform training on the training set, which will later be divided into two subsets: one for building the visual vocabulary and another for training the classifier. Using more samples for training generally results in better performance. However, if computational resources are limited, you may use fewer training images to save time, as long as at least 500 images per class are included.

The system must be tested using the specified subset of test images. Use all 1000 test images (per class) to observe the full performance of the model. Ensure that test images are excluded from training to maintain a fair comparison.

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset, Subset
import numpy as np
import matplotlib.pyplot as plt
import cv2

# Define total train and test sizes
total_train_size = 5000  # Default value for total training images
total_test_size = 1000   # Default value for total test images

# Define batch sizes for DataLoader
train_batch_size = total_train_size
test_batch_size = total_test_size

# Define the number of Visual Words
num_of_visual_words = 1000  # Default value for number of visual words

# Number of classes
num_classes = 5

# Compute images per class for training and testing
images_per_class_train = total_train_size // num_classes  # e.g., 5000 // 5 = 1000 per class
images_per_class_test = total_test_size // num_classes    # e.g., 1000 // 5 = 200 per class

# Define the transformations
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  
])

# Define the class indices for the 5 selected classes: frog, automobile, bird, cat, and deer
selected_classes = [6, 1, 2, 3, 4]  # 6: frog, 1: automobile, 2: bird, 3: cat, 4: deer
class_to_label = {orig_class: new_label for new_label, orig_class in enumerate(selected_classes)}

# Load the CIFAR-10 training set
train_set = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

# Function to filter and remap dataset
def filter_dataset(dataset, images_per_class, selected_classes, class_to_label):
    selected_indices = []
    class_counts = {class_idx: 0 for class_idx in selected_classes}
    remapped_labels = []

    for idx, (image, label) in enumerate(dataset):
        if label in selected_classes and class_counts[label] < images_per_class:
            selected_indices.append(idx)
            remapped_labels.append(class_to_label[label])
            class_counts[label] += 1

            # Stop if we have enough samples for each class
            if all(count >= images_per_class for count in class_counts.values()):
                break

    filtered_dataset = Subset(dataset, selected_indices)
    return filtered_dataset, remapped_labels

# Filter and remap training set
filtered_train_set, train_mapped_labels = filter_dataset(train_set, images_per_class_train, selected_classes, class_to_label)

# Load the CIFAR-10 test set
test_set = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Filter and remap test set
filtered_test_set, test_mapped_labels = filter_dataset(test_set, images_per_class_test, selected_classes, class_to_label)

# Create data loaders for the filtered datasets
train_data_loader = DataLoader(filtered_train_set, batch_size=train_batch_size, shuffle=False)
test_data_loader = DataLoader(filtered_test_set, batch_size=test_batch_size, shuffle=False)

# Extract all training data and remapped labels
train_images, _ = next(iter(train_data_loader))
train_labels = torch.tensor(train_mapped_labels)

train_images = train_images.permute(0, 2, 3, 1)
print(f"Filtered train data: {train_images.shape}")
print(f"Filtered train labels: {train_labels.shape}")

# Extract all test data and remapped labels
test_images, _ = next(iter(test_data_loader))
test_labels = torch.tensor(test_mapped_labels)

test_images = test_images.permute(0, 2, 3, 1)
print(f"Filtered test data: {test_images.shape}")
print(f"Filtered test labels: {test_labels.shape}")

<a id="section-2"></a>
### **Section 2: Keypoint Detection and Feature Extraction (3 points)**

In this section, you will work on detecting keypoints and extracting features from the dataset. Your task is to use **two different feature extraction techniques** to identify keypoints in the images. Visualize two images from each of the five classes (Frog, Automobile, Bird, Cat, Deer) for both feature extraction techniques. For each image, draw circles around the detected keypoints that represent their size.

This step is essential to understand how different feature extractors behave across various classes, setting the foundation for further analysis and classification in later steps.

**Hint:** You can use the OpenCV library to detect keypoints and extract features. You can also upscale the images to improve the visualization of the keypoints.

In [ ]:
def denormalize_image(img_tensor):
    """Convert normalized tensor image to uint8 numpy array"""
    img = img_tensor.numpy()
    img = (img * 0.5 + 0.5) * 255  # Denormalize from [-1, 1] to [0, 255]
    img = np.clip(img, 0, 255).astype(np.uint8)
    return img


def detect_keypoints_and_extract(image, detector):
    """
    Detect keypoints and extract descriptors
    
    Args:
        image: input image 
        detector: feature detector object 
    """
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    keypoints, descriptors = detector.detectAndCompute(gray, None)
    return keypoints, descriptors


def visualize_keypoints(images, labels, class_names, detector_name, detector, num_images_per_class=2, upscale_factor=4):
    """
    Visualize keypoints for selected images from each class
    
    Args:
        images: tensor of images
        labels: tensor of labels
        class_names: list of class names
        detector_name: name of the detector
        detector: detector object
        num_images_per_class: number of images to show per class
        upscale_factor: factor to upscale images for better visualization
    """

    fig, axes = plt.subplots(num_classes, num_images_per_class, figsize=(6*num_images_per_class, 6*num_classes))

    if num_images_per_class == 1: axes = axes.reshape(-1, 1)

    fig.suptitle(f'Keypoint Detection with {detector_name}', fontsize=16, y=0.995)
    
    for class_idx in range(num_classes):
        # Get indices for the first num_images_per_class images of current class
        class_mask = labels == class_idx # <- mask
        class_indices = torch.where(class_mask)[0]
        selected_indices = class_indices[:num_images_per_class]
        
        for img_idx, idx in enumerate(selected_indices):
            img_tensor = images[idx]
            img = denormalize_image(img_tensor)
            
            # upscale
            upscaled_img = cv2.resize(img, None, fx=upscale_factor, fy=upscale_factor, interpolation=cv2.INTER_CUBIC)
            
            # detect keypoints
            keypoints, _ = detect_keypoints_and_extract(upscaled_img, detector)
            
            # draw keypoints 
            img_with_keypoints = cv2.drawKeypoints(
                upscaled_img,
                keypoints, 
                None,
                flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS,
                color=(0, 255, 0)
            )
            

            ax = axes[class_idx, img_idx]
            ax.imshow(img_with_keypoints)
            ax.set_title(f'{class_names[class_idx]}\n{len(keypoints)} keypoints',  fontsize=12)
            ax.axis('off')
    
    plt.tight_layout()
    plt.show()
    
    return fig


In [ ]:

label_to_class_name = {
    0: 'Frog',
    1: 'Automobile',
    2: 'Bird',
    3: 'Cat',
    4: 'Deer'
}
class_names = [label_to_class_name[i] for i in range(len(label_to_class_name))]


# 1. SIFT

print("\n" + "="*60)
print("SIFT")
print("="*60)
print()

# Initialize detector
sift = cv2.SIFT_create()

# Visualize keypoints
fig1 = visualize_keypoints(
    train_images,
    train_labels,
    class_names,
    'SIFT',
    sift,
    num_images_per_class=2,
    upscale_factor=4
)


# 2. ORB

print("\n" + "="*60)
print("ORB")
print("="*60)

# Initialize detector
orb = cv2.ORB_create(nfeatures=1000)  

# Visualize ORB keypoints
fig2 = visualize_keypoints(
    train_images, 
    train_labels, 
    class_names,
    'ORB',
    orb,
    num_images_per_class=2,
    upscale_factor=4
)


In [ ]:
def compute_keypoint_stats(images, labels, class_names, detector, upscale_factor=4):
    """
    Compute average number of keypoints per class
    """

    stats = {name: [] for name in class_names}
    
    for idx in range(len(images)):
        img = denormalize_image(images[idx])
        scaled_img = cv2.resize(img, None, fx=upscale_factor, fy=upscale_factor, interpolation=cv2.INTER_CUBIC)
        keypoints, _ = detect_keypoints_and_extract(scaled_img, detector)
        class_name = class_names[labels[idx].item()]
        stats[class_name].append(len(keypoints))
    
    for class_name in class_names:
        avg_kp = np.mean(stats[class_name])
        std_kp = np.std(stats[class_name])
        print(f"  {class_name:12s}: {avg_kp:6.2f} ± {std_kp:5.2f} keypoints")
    
    return stats

In [ ]:
# Compute statistics for both detectors
sample_size = 100 

sample_indices = []
for class_idx in range(num_classes):
    class_mask = train_labels == class_idx
    class_indices = torch.where(class_mask)[0][:sample_size]
    sample_indices.extend(class_indices.tolist())

sample_images = train_images[sample_indices]
sample_labels = train_labels[sample_indices]


print("SIFT statistics:")
stats_sift = compute_keypoint_stats(sample_images, sample_labels, class_names, sift, 4)
print("\nORB statistics:")
stats_orb = compute_keypoint_stats(sample_images, sample_labels, class_names, orb, 4)



<a id="section-3"></a>
### **Section 3: Building the Visual Vocabulary (3 points)**

In this section, the task is to create a visual vocabulary by clustering feature descriptors extracted from the images using K-Means. Each cluster center in this vocabulary will represent a visual word. Use the two different extraction techniques you implemented to extract descriptors from a subset of training images that includes all categories, and then apply K-Means clustering to build the vocabulary. The number of clusters is fixed at 1000, but you can experiment with different values when you are tuning the hyperparameters in section 9.

To examine the effect of different amounts of training data, build separate visual vocabularies using 30%, 40%, and 50% subsets of the training images. For faster clustering, the `faiss` library can be used, as it provides an efficient implementation of K-Means. Then, visualize the first 10 clusters for each feature extraction technique and each subset size using PCA to reduce the dimensions to 2D.

**Hints:**
1. Begin by debugging the code with a small number of input images to ensure it functions correctly before running it on larger datasets.
2. If the `faiss` library is not available, K-Means clustering can also be performed using the `sklearn` or `scipy` libraries.
3. For visualization, use PCA from `sklearn.decomposition` to reduce the high-dimensional descriptors to 2D. Display up to 10 clusters in the scatter plot to maintain clarity.

In [ ]:
from sklearn.cluster import MiniBatchKMeans 
from sklearn.decomposition import PCA

data_percentages = [0.3, 0.4, 0.5]  # 30%, 40%, 50% of training data

upscale_factor = 4 


def build_visual_vocabulary(descriptors, n_clusters=1000):
    """
    Build visual vocabulary via k-means clustering
    
    Args:
        descriptors: numpy array of descriptors
        n_clusters: number of clusters 
    
    Returns:
        kmeans: fitted K-Means model
        cluster_centers: cluster centers (visual words)
    """

    kmeans = MiniBatchKMeans(
        n_clusters=n_clusters,
        random_state=42,
        max_iter=100,
        verbose=0,
        n_init=4,
        batch_size=1000
    )
    
    kmeans.fit(descriptors)
    
    return kmeans, kmeans.cluster_centers_


def extract_all_descriptors(images, detector, subset_percentage, upscale_factor=4):
    """
    Extract descriptors from a subset of images
    
    Args:
        images: tensor of images
        detector: feature detector 
        subset_percentage: percentage training data to use 
        upscale_factor: factor to upscale images
    
    Returns:
        all_descriptors: numpy array of all descriptors
        num_images_used: number of images processed
    """
    num_images = int(len(images) * subset_percentage)
    all_descriptors = []
    images_with_descriptors = 0
    
    print(f"Extracting descriptors from {num_images} images...")
    
    for idx in range(num_images):
        # Denormalize image
        img = denormalize_image(images[idx])
        
        # Detect keypoints and extract descriptors
        upscaled_img = cv2.resize(img, None, fx=upscale_factor, fy=upscale_factor, interpolation=cv2.INTER_CUBIC)
        _, descriptors = detect_keypoints_and_extract(upscaled_img, detector)
        
        if descriptors is not None and len(descriptors) > 0:
            all_descriptors.append(descriptors)
            images_with_descriptors += 1
        

    if len(all_descriptors) > 0: all_descriptors = np.vstack(all_descriptors)
    else: all_descriptors = np.array([])
    
    print(f"  Total descriptors extracted: {len(all_descriptors)}")
    print(f"  Images with descriptors: {images_with_descriptors}/{num_images}")
    
    return all_descriptors, num_images



def visualize_clusters_pca(descriptors, labels, cluster_centers, detector_name, subset_percentage, n=10):
    """
    Visualize clusters using PCA
    
    Args:
        descriptors: descriptors
        labels: cluster labels for each descriptor
        cluster_centers: cluster centers
        detector_name: name of the feature detector
        subset_percentage: percentage of training data used
        n: number of clusters to display (10)
    """
    print(f"visualizing first {n} clusters using PCA...")

    pca = PCA(n_components=2, random_state=42)
    
    # get only first n clusters
    mask = labels < n
    descriptors_subset = descriptors[mask]
    labels_subset = labels[mask]

    # transform descriptors and cluster centers to 2D
    descriptors_pca = pca.fit_transform(descriptors_subset)
    centers_pca = pca.transform(cluster_centers[:n])
    
    plt.figure(figsize=(12, 8))
    
    colors = plt.cm.tab10(np.linspace(0, 1, n))
    for cluster_id in range(n):
        cluster_mask = labels_subset == cluster_id
        plt.scatter(
            descriptors_pca[cluster_mask, 0],
            descriptors_pca[cluster_mask, 1],
            c=[colors[cluster_id]],
            label=f'Cluster {cluster_id}',
            alpha=0.6,
            s=20
        )
    
    plt.scatter(
        centers_pca[:, 0],
        centers_pca[:, 1],
        c='black',
        marker='X',
        s=200,
        edgecolors='white',
        linewidths=2,
        label='Cluster Centers',
        zorder=10
    )

    plt.title(f'Visual Vocabulary ({int(subset_percentage*100)}% training data, {detector_name})\n')
    plt.xlabel('First Principal Component')
    plt.ylabel('Second Principal Component')
    plt.legend(loc='best', fontsize=9, ncol=2)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()


In [ ]:
vocabularies = {
    'SIFT': {},
    'ORB': {}
}




for detector_name, detector in [('SIFT', sift), ('ORB', orb)]:
    print("\n" + "="*70)
    print(f"Building vocabulary for {detector_name}...")
    print("="*70)
    
    for percentage in data_percentages:
        print(f"Processing for {int(percentage*100)}% of training data")
        
        # extract descriptors
        voc_descriptors, voc_num_images = extract_all_descriptors(
            train_images, detector, percentage, upscale_factor
        )
        
        if len(voc_descriptors) == 0:
            print(f"No descriptors extracted for {detector}")
            continue

        voc_descriptors = voc_descriptors.astype(np.float32)
        
        # build vocabulary
        kmeans, cluster_centers = build_visual_vocabulary(
            voc_descriptors, 
            n_clusters=num_of_visual_words
        )
        
        # Store vocabulary
        vocabularies[detector_name][percentage] = {
            'kmeans': kmeans,
            'cluster_centers': cluster_centers,
            'num_descriptors': len(voc_descriptors),
            'num_images': voc_num_images
        }
        
        # predict cluster labels for visualization
        labels = kmeans.predict(voc_descriptors)
        
        # Visualize clusters
        visualize_clusters_pca(
            voc_descriptors,
            labels,
            cluster_centers,
            detector_name,
            percentage,
            n=10
        )

# Summary of vocabularies built
print("\n" + "="*70)
print("SUMMARY")
print("="*70)

for detector in ['SIFT', 'ORB']:
    print(f"\n{detector}:")
    for percentage in data_percentages:
        if percentage in vocabularies[detector]:
            vocab = vocabularies[detector][percentage]
            print(f"  {int(percentage*100)}% data: "
                  f"{vocab['num_descriptors']:,} descriptors from "
                  f"{vocab['num_images']} images → "
                  f"{num_of_visual_words} visual words")




<a id="section-4"></a>
### **Section 4: Encoding Train Image Features (3 points)**

In this section, the task is to encode image features using the visual vocabulary created earlier. Each image will be represented as a histogram of visual words, reflecting the frequency of each visual word in the image. This representation will allow for comparing images based on their visual content.

To encode an image, identify the nearest visual word (cluster center) for each feature descriptor extracted from the image. Construct a histogram that counts the occurrences of each visual word within the image. The final output will be a collection of histograms, one for each image, where each histogram serves as the feature representation of that image. Once again,  Use the two different extraction techniques you implemented to extract descriptors from the images. Then, encode the images using the visual vocabulary created in the previous step.

**Hint:** Utilize the `faiss` library for efficient nearest neighbor search when assigning each descriptor to the nearest cluster center in the visual dictionary. If `faiss` is not available, consider using other libraries, such as `scikit-learn`, for this step. Once the histograms are obtained, they will be used for further tasks, such as training a classifier. For now, perform the encoding only for the training set.

In [ ]:
def encode_image_bovw(image, detector, vocabulary, upscale_factor=4):
    """
    Encode a single image as a histogram of visual words 
    
    Args:
        image: normalized tensor image
        detector: feature detector
        vocabulary: k-means dictionary
        upscale_factor: upscale factor
    
    Returns:
        histogram: BoVW histogram
    """
    
    img = denormalize_image(image)
    
    # 1) extract descriptors
    img = cv2.resize(img, None, fx=upscale_factor, fy=upscale_factor, interpolation=cv2.INTER_CUBIC)      
    _, descriptors = detect_keypoints_and_extract(img, detector)

    # 2) init histogram
    num_words = len(vocabulary['cluster_centers'])
    histogram = np.zeros(num_words, dtype=np.float32)
    
    if descriptors is None or len(descriptors) == 0: return histogram

    descriptors = descriptors.astype(np.float32)

    # 3) predict visual words of the image 
    kmeans = vocabulary['kmeans']
    labels = kmeans.predict(descriptors)

    # 4) create histogram
    for label in labels: histogram[label] += 1
    
    # 5) normalization
    if histogram.sum() > 0: histogram = histogram / histogram.sum()
    
    return histogram



def encode_dataset_bovw(images, labels, detector, vocabulary, detector_name, upscale_factor=4):
    """
    Encode all images in dataset as BoVW histograms
    
    Args:
        images: tensor of images
        labels: tensor of labels
        detector: feature detector
        vocabulary: vocabulary dictionary
        detector_name: name of detector (for printing)
        upscale_factor: factor to upscale images

    Returns:
        bovw_features: numpy array of shape (num_images, num_visual_words)
        valid_indices: indices of images with features (some might have no keypoints)
    """
    num_images = len(images)

    print(f"Encoding {num_images} images using {detector_name}...")

    bovw_features = []
    valid_indices = []

    for idx in range(num_images):
        # Encode image
        hist = encode_image_bovw(images[idx], detector, vocabulary, upscale_factor)
        if hist.sum() > 0:
            bovw_features.append(hist)
            valid_indices.append(idx)

    
    bovw_features = np.array(bovw_features, dtype=np.float32)
    valid_indices = np.array(valid_indices)

    print(f"Valid images: {len(valid_indices)}/{num_images}")
    print(f"Feature shape: {bovw_features.shape}")

    return bovw_features, valid_indices
    
  

def visualize_bovw_histograms(bovw_features, labels, class_names, detector_name, num_samples=3, n=50):
    """
    Visualize BoVW histograms 
    
    Args:
        bovw_features: BoVW feature vectors (one histogram for each img)
        labels: image labels 
        class_names: list of class names
        detector_name: name of detector
        num_samples: number of samples per class to show
        num_words_to_show: number of visual words to display in histogram
    """

    num_classes = len(class_names)
    
    fig, axes = plt.subplots(num_classes, num_samples, figsize=(15, 10))
    fig.suptitle(f'BoVW - {detector_name}')

    for class_idx in range(num_classes):
        class_mask = labels == class_idx
        class_indices = np.where(class_mask)[0]
        
        # get samples
        if len(class_indices) >= num_samples: selected = np.random.choice(class_indices, num_samples, replace=False)
        else: selected = class_indices[:num_samples]
        
        for sample_idx, img_idx in enumerate(selected):
            ax = axes[class_idx, sample_idx]
            hist = bovw_features[img_idx]
     
            ax.bar(range(n), hist[:n], color='steelblue', alpha=0.7)
            
    
            if sample_idx == 0: ax.set_ylabel(class_names[class_idx], fontsize=11, fontweight='bold')
            if class_idx == 0: ax.set_title(f'Sample {sample_idx + 1}', fontsize=10)
            if class_idx == num_classes - 1: ax.set_xlabel('Visual Word', fontsize=9)
            
            ax.set_ylim([0, hist[:n].max() * 1.2 if hist[:n].max() > 0 else 0.1])
            ax.tick_params(labelsize=8)
            ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

In [ ]:
encoded_train_features = {
    'SIFT': {},
    'ORB': {}
}


for detector_name, detector in [('SIFT', sift), ('ORB', orb)]:
    print("\n" + "="*70)
    print(f"Encoding training images - {detector_name}")
    print("="*70)
    
    for percentage in data_percentages:
        print(f"\n{'*'*70}")
        print(f"Processing {int(percentage*100)}% vocabulary")
        print(f"\n{'*'*70}")
        
        if percentage not in vocabularies[detector_name]: continue
        
        # get vocabulary
        vocabulary = vocabularies[detector_name][percentage]

        # encode all training images 
        bovw, valid_indices = encode_dataset_bovw(
            train_images,
            train_labels,
            detector,
            vocabulary,
            detector_name,
            upscale_factor
        )

        encoded_train_features[detector_name][percentage] = {
            'features': bovw,
            'labels': train_labels[valid_indices],
            'valid_indices': valid_indices
        }

       
        # visualize histograms
        visualize_bovw_histograms(
            bovw,
            train_labels[valid_indices],
            [label_to_class_name[i] for i in range(len(label_to_class_name))],
            f"{detector_name} ({int(percentage*100)}% vocab)",
            num_samples=3,
            n=200
        )

In [ ]:
# Sparsity analysis
print("\n" + "="*70)
print("Sparsity Analysis of BoVW Histograms")
for detector_name in ['SIFT', 'ORB']:
    for percentage in data_percentages:
        if percentage in encoded_train_features[detector_name]:
            features = encoded_train_features[detector_name][percentage]['features']
            labels = encoded_train_features[detector_name][percentage]['labels']

            non_empty_count = (features.sum(axis=1) > 0).sum()
            total_count = len(features)
            
            print(f"\n{detector_name} {int(percentage*100)}%:")
            print(f"Non-empty histograms: {non_empty_count}/{total_count} ({non_empty_count/total_count*100:.1f}%)")


            for class_idx in range(num_classes):
                class_mask = labels == class_idx
                class_features = features[class_mask]
                non_empty_in_class = (class_features.sum(axis=1) > 0).sum()
                min_number_of_visual_words_for_class = np.min((class_features > 0).sum(axis=1)) 

                print(f"{class_names[class_idx]}: {non_empty_in_class}/{class_mask.sum()} non-empty, min number of visual words: {min_number_of_visual_words_for_class}")


<a id="section-5"></a>
### **Section 5: Visualizing the Bag of Visual Words for Each Class (3 points)**

In this section, the task is to visualize the Bag of Visual Words for each class using the histograms generated in the previous step. The goal is to plot the mean histogram of visual words for each class, showing the distribution of visual words across the different categories in the training set.

Use the two different extraction techniques you implemented for this visualization. For each technique, calculate the mean histogram for each class and create a bar plot to display these histograms. Ensure that the plots are labeled clearly with the class names and feature descriptor types.

In [ ]:
def visualize_mean_bovw_per_class(features, labels, class_names, detector_name, percentage):
    """
    Visualize mean BoVW histogram for each class
    
    Args:
        features: feature vectors (num_images, num_visual_words)
        labels: image labels
        class_names: list of class names
        detector_name: name of detector
        percentage: percentage of training data used
    """

    num_classes = len(class_names)
    num_visual_words = features.shape[1]
    

    mean_histograms = [] # one per class

    for class_idx in range(num_classes):
        class_mask = labels == class_idx
        mean_histograms.append(features[class_mask].mean(axis=0))
    
    mean_histograms = np.array(mean_histograms) # (num_classes, num_visual_words)
    
    
    fig, axes = plt.subplots(num_classes, 1, figsize=(15, 3*num_classes))
    if num_classes == 1: axes = [axes]

    fig.suptitle(f'Mean BoVW per Class - {detector_name} ({int(percentage*100)}%)', fontsize=14, fontweight='bold')
    
    for class_idx in range(num_classes):
        ax = axes[class_idx]
    
        ax.bar(range(num_visual_words), mean_histograms[class_idx], color="green", alpha=0.4, width=1.1)
        
        ax.set_ylabel('Mean Frequency')
        ax.set_title(f'{class_names[class_idx]}', fontweight='bold', loc='left')
        ax.grid(True, alpha=0.3, axis='y')
        ax.set_xlabel('Visual Word')
    
    plt.tight_layout()
    plt.show()



for detector_name in ['SIFT', 'ORB']:
    print(f"\n{'='*70}")
    print(f"{detector_name}")
    print(f"{'='*70}")
    
    for percentage in data_percentages:
        if percentage in encoded_train_features[detector_name]:
            visualize_mean_bovw_per_class(
                features = encoded_train_features[detector_name][percentage]['features'],
                labels = encoded_train_features[detector_name][percentage]['labels'],
                class_names = class_names,
                detector_name = detector_name,
                percentage = percentage
            )

<a id="section-6"></a>
### **Section 6: Encoding Test Image Features (0 points)**

In this section, the task is to encode the test image features using the visual vocabulary created from the training set. Similar to the previous encoding step, each test image will be represented as a histogram of visual words, which will then be used for evaluating classification performance.

Use the same two feature extraction techniques you selected earlier. Extract keypoints and descriptors for the test images, then encode these images using the visual vocabulary. This will allow you to compare the encoded features of test images against those of the training set.

**Hint:** Reuse the functions developed earlier for extracting keypoints, descriptors, and encoding images. Ensure that you use the visual vocabulary constructed with the training images for consistency.

In [ ]:
encoded_test_features = {
    'SIFT': {},
    'ORB': {}
}

for detector_name, detector in [('SIFT', sift), ('ORB', orb)]:
    print("\n" + "="*70)
    print(f"Encoding test images - {detector_name}")
    print("="*70)
    
    for percentage in data_percentages:
        
        vocabulary = vocabularies[detector_name][percentage]

        # encode all test images 
        bovw, valid_indices = encode_dataset_bovw(
            test_images,
            test_labels,
            detector,
            vocabulary,
            detector_name,
            upscale_factor
        )

        encoded_test_features[detector_name][percentage] = {
            'features': bovw,
            'labels': test_labels[valid_indices],
            'valid_indices': valid_indices
        }



<a id="section-7"></a>
### **Section 7: Training the Classifiers (5 points)**

In this section, the task is to create two one-vs-rest (OvR) SVM classifiers using the 50% of the training data that was **not** used for creating the visual dictionary. This ensures that the classifiers are trained on a different subset of data, providing a more robust evaluation of the visual vocabulary's effectiveness.

For each of the two selected feature extraction techniques, create one-vs-rest classifiers for all classes. For now, use default parameter values when training the classifiers; you will experiment with different hyperparameters in later steps.

**Note:** Training an OvR classifier can take around 5 to 7 minutes. Therefore, it's advisable to first test your code with a smaller subset of the training data to verify that your implementation works correctly before running it on the full dataset.

In [ ]:
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier

# classifiers
classifiers = {
    'SIFT': OneVsRestClassifier(SVC(kernel='linear')),
    'ORB': OneVsRestClassifier(SVC(kernel='linear'))
}

for detector_name, clf in classifiers.items():
    print(f"Training {detector_name} classifier...")

    train_x = encoded_train_features[detector_name][0.5]['features']
    train_y = encoded_train_features[detector_name][0.5]['labels']

    # we have to discard the first 50% of training data as we did use it for building the vocabulary
    train_x = train_x[ (len(train_x) // 2) : ]
    train_y = train_y[ (len(train_y) // 2) : ]

    print(f"Training data shape: {train_x.shape}, Training labels shape: {train_y.shape}")

    print("Fitting classifier...")

    clf.fit(train_x, train_y)

    print(f"{detector_name} training complete!")
    

    y_train_pred = clf.predict(train_x)
    train_acc = (y_train_pred == np.array(train_y)).mean()
    print(f"Training accuracy: {train_acc*100:.2f}%")

<a id="section-8"></a>
### **Section 8: Evaluating the Classifiers (12 points)**

In this section, you will evaluate the performance of your one-vs-rest (OvR) SVM classifiers on the test data. The goal is to classify each test image using each binary classifier and rank the images based on the classification scores, resulting in a ranked list of images for each class. Ideally, images belonging to the target class should appear at the top of the respective list. To conduct this evaluation, use the test image histograms generated earlier for the two selected feature extraction techniques. Classify each test image with each classifier, rank them based on their confidence scores, and then compute the Mean Average Precision (mAP) across all classes. The mAP for a single class $c$ is defined as:

$
\text{mAP}_c = \frac{1}{m_c} \sum_{i=1}^{n} \frac{f_c(x_i)}{i}
$

where:
- $n$ is the total number of images ($n = 50 \times 5 = 250$),
- $m_c$ is the number of images of class $c$ ($m_c = 50$),
- $x_i$ is the $i^{th}$ image in the ranked list $X = \{ x_1, x_2, \dots, x_n \}$,
- $f_c$ is a function that returns the number of images of class $c$ in the first $i$ images if $x_i$ is of class $c$, and 0 otherwise.

For instance, if you are retrieving images of class "R" and the sequence of ranked images is $[R, R, T, R, T, T, R, T]$, then $n = 8$, $m_c = 4$, and:

$
AP = \frac{1}{4} \left( \frac{1}{1} + \frac{2}{2} + \frac{0}{3} + \frac{3}{4} + \frac{0}{5} + \frac{0}{6} + \frac{4}{7} + \frac{0}{8} \right).
$

In addition to the quantitative analysis, perform a qualitative analysis by visualizing the top-5 and bottom-5 ranked test images.

In [ ]:
def calc_AP(ground_truth, scores, target_class):
    """
    Compute Average Precision for a single class
    
    Args:
        ground_truth: true labels 
        scores: confidence scores for target_class
        target_class: index of the class to compute AP for
    
    Returns:
        ap: Average Precision for this class
    """
    # sort images by confidence     
    sorted_indices = np.argsort(scores)  # get sorted indices
    sorted_indices = sorted_indices[::-1].copy()  # descending order
    sorted_labels = ground_truth[sorted_indices]  # sort labels by indices

    # number of images of target class
    m_c = np.sum(ground_truth == target_class)
    if m_c == 0: return 0.0
    

    ap = 0.0
    num_correct = 0
    
    for i, label in enumerate(sorted_labels, start=1):
        if label == target_class:
            num_correct += 1
            ap += num_correct / i
    
    ap *= 1 / m_c
    
    return ap



def calc_mAP(ground_truth, scores, num_classes):
    """
    Compute mean Average Precision over all classes
    
    Args:
        ground_truth: true labels
        scores: confidence scores for each class (num_images, num_classes)
        num_classes: number of classes
    
    Returns:
        mAP: mean Average Precision
        APs: list of AP for each class
    """
    APs = []
    
    for class_idx in range(num_classes):
        ap = calc_AP(ground_truth, scores[:, class_idx], class_idx)
        APs.append(ap)
    
    mAP = np.mean(APs)
    
    return mAP, APs

In [ ]:
def visualize_top_bottom_ranked_images(n, decision_scores, ground_truth, detector_name):
    """
    Visualize top-n and bottom-n ranked images for each class
    Args:
        n: number of top and bottom images to show
        ground_truth: true labels
        decision_scores: confidence scores for each class
        detector_name: name of the detector
    """
    
    # get test images
    valid_test_images = test_images[encoded_test_features[detector_name][0.5]['valid_indices']]

    for class_idx in range(num_classes):

        # sort images by confidence
        class_scores = decision_scores[:, class_idx]  # get scores for this class
        sorted_indices = np.argsort(class_scores)  # get sorted indices
        sorted_indices = sorted_indices[::-1].copy()  # descending order

        fig, axes = plt.subplots(2, n, figsize=(15, 6))

        fig.suptitle(f'{class_names[class_idx]} - {detector_name}\nTop {n} and bottom {n} (right) ranked images', fontweight='bold')

        # top-n images 
        for i in range(n):
            index = sorted_indices[i]

            img = denormalize_image(valid_test_images[index])
            true_label = ground_truth[index]

            axes[0, i].imshow(img)
            axes[0, i].axis('off')
            
        
            color = 'green' if true_label == class_idx else 'red'
            axes[0, i].set_title(f'Rank {i+1}\n{class_names[true_label]}', color=color, fontweight='bold')

        # last n images 
        for i in range(n):
            index = sorted_indices[-(n-i)]
            img = denormalize_image(valid_test_images[index])
            true_label = ground_truth[index]
            
            axes[1, i].imshow(img)
            axes[1, i].axis('off')
            
            color = 'green' if true_label == class_idx else 'red'
            axes[1, i].set_title(f'Rank {len(sorted_indices)-(n-i-1)}\n{class_names[true_label]}', color=color, fontweight='bold')


        axes[0, 0].text(-0.3, 0.5, f"top {n}", transform=axes[0, 0].transAxes, verticalalignment='center')
        axes[1, 0].text(-0.3, 0.5, f"bottom {n}", transform=axes[1, 0].transAxes, verticalalignment='center')

        plt.tight_layout()
        plt.show()


In [ ]:
results = {}

for detector_name in ['SIFT', 'ORB']:
    print(f"\n{'='*70}")
    print(f"Evaluating {detector_name} Classifier")
    print(f"{'='*70}")
    
    # Get test data
    test_x =  np.array(encoded_test_features[detector_name][0.5]['features'])
    test_y = np.array(encoded_test_features[detector_name][0.5]['labels'])
    
    # Get trained classifier
    clf = classifiers[detector_name]
    
    # Get decision scores for ranking
    decision_scores = clf.decision_function(test_x)
    
    print(f"test set: {test_x.shape[0]} images")
    print(f"decision scores shape: {decision_scores.shape}")
    
    # Compute mAP
    mAP, APs = calc_mAP(test_y, decision_scores, num_classes=num_classes)
    results[detector_name] = {
        "mAP": mAP,
        "APs": APs
    }

    print(f"\n{detector_name} mAP: {mAP*100:.2f}%")
    for class_idx, ap in enumerate(APs):
        print(f"Class {class_idx} AP: {ap*100:.2f}%")


    visualize_top_bottom_ranked_images(5, decision_scores, test_y, detector_name)

<a id="section-9"></a>
### **Section 9: Hyperparameter Search (16 points)**

In this section, the task is to perform an extensive hyperparameter search to optimize the performance of your classifiers. You will experiment with various parameters, including the number of visual words (e.g., 500, 1000, 1500), different training subset sizes (e.g., 30%, 40%, 50%), SVM parameters (e.g., kernel types like 'linear' or 'rbf', regularization parameter $C$ values such as 0.1, 1, 10, and gamma settings like 'scale' or specific values such as 0.01, 0.001), and settings of the feature extractors (e.g., the number of keypoints or scale levels). Start by testing your code on the smallest subset to ensure it functions correctly before proceeding with a full hyperparameter search. Once validated, conduct the search using larger subsets and systematically iterate through the different parameter combinations, potentially using nested loops or grid search. Be sure to record the performance results for each combination to identify the best settings based on metrics like the Mean Average Precision (mAP).

In [ ]:
# feature extractor parameters
upscale_factor_options = [2, 4, 8]
detector_types = ['SIFT', 'ORB']
nfeatures_options = [500, 1000]

# visual vocabulary parameters
num_visual_words_options = [500, 1000, 1500]
training_subset_sizes = [0.3, 0.4, 0.5]


# support vector machine parameters
svm_kernels = ['linear', 'rbf']
svm_C_values = [0.1, 1, 10]
svm_gamma_values = ['scale', 0.01, 0.001]


In [ ]:
class FitAndEvaluationParams:
    def __init__(self, detector_type, nfeatures, upscale_factor, num_visual_words, training_subset_size, svm_kernel, svm_C, svm_gamma):
        self.detector_type = detector_type
        self.nfeatures = nfeatures
        self.upscale_factor = upscale_factor
        self.num_visual_words = num_visual_words
        self.training_subset_size = training_subset_size
        self.svm_kernel = svm_kernel
        self.svm_C = svm_C
        self.svm_gamma = svm_gamma

    def __str__(self):
        return (f"Detector: {self.detector_type}, nfeatures: {self.nfeatures}, "
                f"Upscale: {self.upscale_factor}, Visual Words: {self.num_visual_words}, "
                f"Subset Size: {self.training_subset_size}, SVM Kernel: {self.svm_kernel}, "
                f"C: {self.svm_C}, Gamma: {self.svm_gamma}")


class FitAndEvaluationTestResult:
    def __init__(self, params):
        self.params = params
        self.mAP = None
        self.test_acc = None
        self.training_time = None
        self.vocabulary_time = None


    def set_results(self, mAP, test_acc, training_time, vocabulary_time):
        self.mAP = mAP
        self.test_acc = test_acc
        self.training_time = training_time
        self.vocabulary_time = vocabulary_time
    
     


In [ ]:
import time

def perform_fit_and_evaluation_test(params):
    """
    Fit and evaluate BoVW + SVM classifier with given parameters

    args:
        params: FitAndEvaluationParams 
    """

    upscale_factor, detector_type, nfeatures, num_visual_words, vocabulary_percentage, svm_kernel, svm_C, svm_gamma = (
        params.upscale_factor,
        params.detector_type,
        params.nfeatures,
        params.num_visual_words,
        params.training_subset_size,
        params.svm_kernel,
        params.svm_C,
        params.svm_gamma
    )


    print("\n" + "="*70)
    print("Testing configuration:")
    print(params)
    print("="*70)

    # 1) Initialize detector 
    if detector_type == 'SIFT': detector = cv2.SIFT_create(nfeatures=nfeatures)
    elif detector_type == 'ORB': detector = cv2.ORB_create(nfeatures=nfeatures)
    else: raise ValueError("detector_type must be 'SIFT' or 'ORB'")

    # 2) Upscale images and extract descriptors from training data for vocabulary creation
    start_vocab_time = time.time()
    voc_descriptors, voc_num_images = extract_all_descriptors(
        train_images, detector, vocabulary_percentage, upscale_factor
    )

    if len(voc_descriptors) == 0: raise ValueError("No descriptors extracted from training data")

    voc_descriptors = voc_descriptors.astype(np.float32)

    # 3) Build visual vocabulary
    k_means, visual_words = build_visual_vocabulary(
        voc_descriptors, 
        n_clusters=num_visual_words
    )

    vocabulary = {
        'kmeans': k_means,
        'cluster_centers': visual_words,
        'num_descriptors': len(voc_descriptors),
        'num_images': voc_num_images
    }

    end_vocab_time = time.time()
    vocabulary_time = end_vocab_time - start_vocab_time

    print(f"\nVocabulary built with {len(voc_descriptors):,} descriptors from {voc_num_images} images. Creation time: {vocabulary_time:.2f} seconds")


    # 4) Encode training images
    # We have to consider only the second part of training data as the first percentage% was used for building the vocabulary
    # skip the images used for vocabulary creation (percentage% of training data)
    start_index = int(len(train_images) * vocabulary_percentage)
    remaining_train_images = train_images[start_index:]
    remaining_train_labels = train_labels[start_index:]


    train_x, train_valid_indices = encode_dataset_bovw(
        remaining_train_images,
        remaining_train_labels,
        detector,
        vocabulary,
        detector_type,
        upscale_factor
    )

    train_y = remaining_train_labels[train_valid_indices]

    # 5) Train SVM classifier
    print("Fitting classifier...")
    print(f"Training data shape: {train_x.shape}, Training labels shape: {train_y.shape}")


    if svm_kernel == 'linear': clf = OneVsRestClassifier(SVC(kernel=svm_kernel, C=svm_C, probability=False))
    else: clf = OneVsRestClassifier(SVC(kernel=svm_kernel, C=svm_C, gamma=svm_gamma, probability=False))


    startTime = time.time()

    clf.fit(train_x, train_y)

    endTime = time.time()
    training_time = endTime - startTime
    print(f"Training complete in {training_time:.2f} seconds")

    # 6) Accuracy evaluation on test data
    test_x, test_valid_indices = encode_dataset_bovw(
        test_images,
        test_labels,
        detector,
        vocabulary,
        detector_type,
        upscale_factor
    )

    test_y = test_labels[test_valid_indices]
    test_y = np.array(test_y)

    y_test_pred = clf.predict(test_x)
    test_acc = (y_test_pred == np.array(test_y)).mean()
    print(f"Test accuracy: {test_acc*100:.2f}%")


    # 7) mAP evaluation on test data
    decision_scores = clf.decision_function(test_x)
    mAP, _ = calc_mAP(test_y, decision_scores, num_classes=num_classes)

    print(f"{detector_type} mAP: {mAP*100:.2f}%")

    # Save results
    result = FitAndEvaluationTestResult(params)
    result.set_results(
        mAP=mAP,
        test_acc=test_acc,
        training_time=training_time,
        vocabulary_time=vocabulary_time
    )

    return result


In [ ]:
# Test 
test_params = FitAndEvaluationParams(
    detector_type='SIFT',
    nfeatures=1000,
    upscale_factor=4,
    num_visual_words=1000,
    training_subset_size=0.2,
    svm_kernel='linear',
    svm_C=1,
    svm_gamma='scale'
)

test_result = perform_fit_and_evaluation_test(test_params)

In [ ]:
import datetime
import pandas as pd

def build_dataframe_from_results(results):
    """
    Build pandas DataFrame from results list
    """
    results_data = []
    for r in results:
        results_data.append({
            'detector': r.params.detector_type,
            'nfeatures': r.params.nfeatures,
            'upscale_factor': r.params.upscale_factor,
            'num_visual_words': r.params.num_visual_words,
            'training_subset_size': r.params.training_subset_size,
            'svm_kernel': r.params.svm_kernel,
            'svm_C': r.params.svm_C,
            'svm_gamma': r.params.svm_gamma,
            'mAP': r.mAP,
           # 'test_accuracy': r.test_acc,
            'training_time': r.training_time,
            'vocabulary_time': r.vocabulary_time,
            'total_time': r.training_time + r.vocabulary_time
        })
    
    df = pd.DataFrame(results_data)
    return df


def save_results(results):
    """
    Save hyperparameter search results to CSV 
    """
  
    filename = f"hyperparam_results"
    results = [r for r in results if r.mAP is not None]
    df = build_dataframe_from_results(results)
    csv_filename = f"{filename}.csv"
    df.to_csv(csv_filename, index=False)
    print(f"Results saved to {csv_filename}")
    



def plot_results(df, save_path=None):
    """
    Visualization of hyperparameter search results
    
    Args:
        df: dataframe with results
        save_path: Path to save the figure (default: don't save)
    """
  
    # Create figure with subplots
    fig = plt.figure(figsize=(20, 9))
    
    # 1. mAP by detector type
    ax1 = plt.subplot(3, 4, 1)
    df.boxplot(column='mAP', by='detector', ax=ax1)
    ax1.set_title('mAP by Detector Type')
    ax1.set_xlabel('Detector')
    ax1.set_ylabel('mAP')
    plt.suptitle('')
    
    # 2. mAP by number of features
    ax2 = plt.subplot(3, 4, 2)
    df.boxplot(column='mAP', by='nfeatures', ax=ax2)
    ax2.set_title('mAP by Number of Features')
    ax2.set_xlabel('Number of Features')
    ax2.set_ylabel('mAP')
    plt.suptitle('')
    
    # 3. mAP by upscale factor
    ax3 = plt.subplot(3, 4, 3)
    df.boxplot(column='mAP', by='upscale_factor', ax=ax3)
    ax3.set_title('mAP by Upscale Factor')
    ax3.set_xlabel('Upscale Factor')
    ax3.set_ylabel('mAP')
    plt.suptitle('')
    
    # 4. mAP by visual words
    ax4 = plt.subplot(3, 4, 4)
    df.boxplot(column='mAP', by='num_visual_words', ax=ax4)
    ax4.set_title('mAP by Visual Words')
    ax4.set_xlabel('Number of Visual Words')
    ax4.set_ylabel('mAP')
    plt.suptitle('')
    
    # 5. mAP by training subset size
    ax5 = plt.subplot(3, 4, 5)
    df.boxplot(column='mAP', by='training_subset_size', ax=ax5)
    ax5.set_title('mAP by Training Subset Size')
    ax5.set_xlabel('Training Subset Size')
    ax5.set_ylabel('mAP')
    plt.suptitle('')
    
    # 6. mAP by SVM kernel
    ax6 = plt.subplot(3, 4, 6)
    df.boxplot(column='mAP', by='svm_kernel', ax=ax6)
    ax6.set_title('mAP by SVM Kernel')
    ax6.set_xlabel('SVM Kernel')
    ax6.set_ylabel('mAP')
    plt.suptitle('')
    
    # 7. mAP by SVM C
    ax7 = plt.subplot(3, 4, 7)
    df.boxplot(column='mAP', by='svm_C', ax=ax7)
    ax7.set_title('mAP by SVM C')
    ax7.set_xlabel('SVM C')
    ax7.set_ylabel('mAP')
    plt.suptitle('')
    
    # 8. mAP by SVM gamma
    ax8 = plt.subplot(3, 4, 8)
    df.boxplot(column='mAP', by='svm_gamma', ax=ax8)
    ax8.set_title('mAP by SVM Gamma')
    ax8.set_xlabel('SVM Gamma')
    ax8.set_ylabel('mAP')
    plt.suptitle('')
    
    # 9. Top 10 configurations
    ax9 = plt.subplot(3, 4, 9)
    top10 = df.nlargest(10, 'mAP').reset_index(drop=True)
    bars = ax9.barh(range(len(top10)), top10['mAP'])
    ax9.set_yticks(range(len(top10)))
    ax9.set_yticklabels([f"Config {i+1}" for i in range(len(top10))])
    ax9.set_xlabel('mAP')
    ax9.set_title('Top 10 Configurations by mAP')
    ax9.invert_yaxis()
    for i, bar in enumerate(bars):
        ax9.text(bar.get_width(), bar.get_y() + bar.get_height()/2, 
                f'{top10.iloc[i]["mAP"]:.3f}', 
                va='center', ha='left', fontsize=8)
    

    
    plt.tight_layout()
    
    if save_path: plt.savefig(save_path, dpi=300, bbox_inches='tight')
    
    plt.show()
    
    # Print insights
    print("\n" + "="*80)
    print("KEY INSIGHTS")
    print("="*80)
    
    # Best detector
    best_detector = df.groupby('detector')['mAP'].mean().idxmax()
    print(f"Best detector on average: {best_detector} (mean mAP: {df.groupby('detector')['mAP'].mean().max():.3f})")
    
    # Best kernel
    best_kernel = df.groupby('svm_kernel')['mAP'].mean().idxmax()
    print(f"Best SVM kernel on average: {best_kernel} (mean mAP: {df.groupby('svm_kernel')['mAP'].mean().max():.3f})")
    
    # Best configuration
    best_config = df.loc[df['mAP'].idxmax()]
    print(f"\nBest overall configuration (mAP: {best_config['mAP']:.3f}):")
    print(f"  - Detector: {best_config['detector']}")
    print(f"  - Features: {best_config['nfeatures']}")
    print(f"  - Upscale: {best_config['upscale_factor']}")
    print(f"  - Visual Words: {best_config['num_visual_words']}")
    print(f"  - Training Subset: {best_config['training_subset_size']}")
    print(f"  - Kernel: {best_config['svm_kernel']}")
    print(f"  - C: {best_config['svm_C']}")
    print(f"  - Gamma: {best_config['svm_gamma']}")


# Example usage:
# After your grid search loop:
# df = save_results_to_file(results)
# plot_results(results, save_path='hyperparameter_search_plots.png')

    



In [ ]:
# Hyperparameter grid search
import itertools

param_combinations = list(itertools.product(
    detector_types[0:2],
    nfeatures_options[0:2],
    upscale_factor_options[0:2],
    num_visual_words_options[0:2],
    training_subset_sizes[0:2],
    svm_kernels[0:2],
    svm_C_values[0:2],
    svm_gamma_values[0:2]
))

print(f"\nTotal parameter combinations to test: {len(param_combinations)}")

results = []
for param_tuple in param_combinations:
    params = FitAndEvaluationParams(
        detector_type=param_tuple[0],
        nfeatures=param_tuple[1],
        upscale_factor=param_tuple[2],
        num_visual_words=param_tuple[3],
        training_subset_size=param_tuple[4],
        svm_kernel=param_tuple[5],
        svm_C=param_tuple[6],
        svm_gamma=param_tuple[7]
    )

    try:
        result = perform_fit_and_evaluation_test(params)
        results.append(result)
    except Exception as e:
        print(f"Error with parameters {params}: {e}")


# Save results to CSV
save_results(results)



In [ ]:
save_results(results)
plot_results(build_dataframe_from_results(results), save_path='hyperparameter_search_plots.png')

<a id="section-10"></a>
### **Section 10: Using CLIP for Image Classification (5 points)**

**<span style="color:red">⚠️ NOTE: This section should NOT be included in the report. It is only meant to be completed in the code cells. The purpose of this task is to introduce you to a more state-of-the-art model (CLIP) compared to Bag of Visual Words (BoVW). Vision Transformers (ViT) will be covered in more detail in the Deep Learning 1 course next period!</span>**

In this section, you will use a pre-trained CLIP model for image classification. CLIP (Contrastive Language-Image Pretraining) is a vision-language transformer model trained on a large dataset of images and text. It consists of two main components: a Vision Transformer (ViT) and a text Transformer. The ViT encodes images by dividing them into patches (tokens), flattening each patch into a vector, and passing them through a sequence of Transformer layers to produce an encoded representation of the image.

For this task, you will use the visual transformer component of CLIP to extract encoded representations of the input images. While this is not the typical way to use CLIP (which involves encoding both images and text for similarity comparison), it provides an interesting application of this state-of-the-art model for image classification.

**To Install CLIP:**
```python
pip install git+https://github.com/openai/CLIP.git
```

**Additional Reading (if you're interested):**
- [OpenAI CLIP Overview](https://openai.com/clip)
- [Vision Transformer (ViT) Paper](https://arxiv.org/abs/2010.11929)
- [Tutorial on Vision Transformers](https://d2l.ai/chapter_attention-mechanisms-and-transformers/vision-transformer.html)
- [UvA's Deep Learning Introduction to ViTs](https://uvadlc-notebooks.readthedocs.io/en/latest/tutorial_notebooks/tutorial15/Vision_Transformer.html)

First, create DataLoaders for both the training and test datasets by filtering the CIFAR-10 dataset to include only the selected classes: frog, automobile, bird, cat, and deer. Use a batch size of 16 for both DataLoaders, and resize the images to 224x224 to match the input size requirements for CLIP. Remember to normalize the images using the appropriate mean and standard deviation. Use a training set size of 1000 images per class and a test set size of 200 images per class.

In [ ]:
# YOUR CODE HERE

Next, we load a pre-trained CLIP model, which is a vision-language transformer designed to predict the text that describes an image and vice versa. The model consists of two components: a Vision Transformer (ViT) for encoding images and a text Transformer for encoding text. 

In [ ]:
import clip

# Setup the model and the preprocessor
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load the pre-trained CLIP model
model, preprocess = clip.load("ViT-B/32", device=device)

To extract the visual tokens using the CLIP model, start by initializing two empty lists: one for storing the image features and another for the labels. Use `tqdm` to create a progress bar that tracks the extraction process over the DataLoader. Iterate through the DataLoader, extracting images and labels. Disable gradient computation, and then encode the images with `model.encode_image(images)`. Append the encoded features and labels to their respective lists. Remember, the output from the model will have the shape `(batch_size, 512)` due to batched processing, if your are using a single you should reshape the output to `(512,)` to remove the batch dimension.

In this example, we will use the class token as the visual representation of the image. The class token is a 512-dimensional vector that represents the image. We will use this vector to train a classifier to classify the images.

**Note:** The CLIP model is quite large and may take some time to extract features from the images. You can use the `tqdm` library to create a progress bar that shows the extraction progress. It is recommended to test the code with a smaller subset of images to ensure it functions correctly before running it on the full dataset. With the default batch size of 16 and a training set size of 1000 images, the extraction process may take a 10-20 minutes.

In [ ]:
# YOUR CODE HERE

In [ ]:
# Stack the (batched) visual tokens extracted from the images. The resulting shape will be (number_of_images, 512).
stacked_image_features = torch.cat(image_features_list)
stacked_target_labels = torch.cat(target_labels_list)

To train a classifier using the visual tokens, start by initializing an SVM classifier using `SVC()` from `scikit-learn`. You can play around with different hyperparameters such as kernel type, regularization parameter, and gamma to find the best configuration. Finally, use the `fit` method to train the classifier on the visual image features and labels.

In [ ]:
# YOUR CODE HERE

To evaluate the classifier, start by extracting the visual tokens from the test images using the same method applied to the training set. Loop through the test DataLoader, encode each batch of images using the model (e.g., `model.encode_image(images)`), and store the results in separate lists for the features and labels. After extracting all the features, stack them into a single tensor for both the features and labels. This process will prepare the test data for use in evaluating the classifier's performance.

In [ ]:
# YOUR CODE HERE

To evaluate the performance of your classifier, use the test set features you extracted earlier. First, generate predictions for the test set by passing the stacked test features into your trained classifier's `predict` method. Next, use the `classification_report` function from `sklearn.metrics` to create a detailed report that includes metrics such as precision, recall, and F1-score for each class. Finally, print the report to analyze how well your classifier performs across the different classes.

In [ ]:
# YOUR CODE HERE

<a id="section-x"></a>
### **Section X: Individual Contribution Report *(Mandatory)***

Because we want each student to contribute fairly to the submitted work, we ask you to fill out the textcells below. Write down your contribution to each of the assignment components in percentages. Naturally, percentages for one particular component should add up to 100% (e.g. 30% - 30% - 40%). No further explanation has to be given.

| Name | Contribution on Research | Contribution on Programming | Contribution on Writing |
| -------- | ------- | ------- | ------- |
|  | - % | - % | - % |
|  | - % | - % | - % |
|  | - % | - % | - % |
|  | - % | - % | - % |

### - End of Notebook -